### **Exploring the learning task: Node Classification**

In [ ]:
### NOTE: Uncomment below to mount google drive [Only for Colab environment]
# from google.colab import drive
# drive.mount('/content/drive')

# Define the base path for the runs in this file
import os; base_path = os.getcwd() # Define the base path

In [ ]:
# Install relevant packages for the runs
!pip install torch torch-geometric torch-cluster torch-sparse torch-scatter matplotlib networkx

Experimenting new datasets

In [ ]:
import torch
import numpy as np
import torchvision
import torchvision.transforms as transforms
from typing import Any
import torch_geometric
from torch_geometric.data import Dataset as GraphDataset
from torch_geometric.loader import DataLoader as GraphDataLoader

import os
from typing import Tuple
import numpy as np
import torch
from torchvision import transforms
from torch_geometric.data import Data

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import random
from torch_geometric.utils import to_networkx
import networkx as nx
import matplotlib.pyplot as plt

import os
import pickle
from copy import deepcopy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
"""
Helper functions
"""
def convert_to_networkx(graph, n_sample=None):
    g = to_networkx(graph, node_attrs=["x"])
    y = graph.y.numpy()
    if n_sample is not None:
        sampled_nodes = random.sample(g.nodes, n_sample)
        g = g.subgraph(sampled_nodes)
        y = y[sampled_nodes]

    return g, y


def plot_graph(g, y):
    plt.figure(figsize=(9, 7))
    nx.draw_spring(g, node_size=30, arrows=False, node_color=y)
    plt.show()


# g, y = convert_to_networkx(graph, n_sample=1000)
# plot_graph(g, y)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ChebConv, GraphConv, SGConv, GENConv, GeneralConv, GATv2Conv, TransformerConv, EGConv


# MLP MODEL
class MLP(nn.Module):
    name = "MLP"

    def __init__(self, in_channels:int = -1, hidden_channels:int = -1, out_channels:int = -1):
        super().__init__()
        assert in_channels > 0 and out_channels > 0, f"{MLP.name} model init was given bad args"
        self.lin1 = nn.Linear(in_channels, hidden_channels)
        self.lin2 = nn.Linear(hidden_channels, out_channels)
        self.lin3 = nn.Linear(hidden_channels, out_channels)
        self.lin4 = nn.Linear(hidden_channels, out_channels)
        self.bn1 = torch.nn.BatchNorm1d(num_features=hidden_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        edge_index = torch.stack([edge_index[1], edge_index[0]], dim=0)

        x = F.relu(self.bn1(self.lin1(x)))
        output = F.relu(self.lin2(x)) + F.relu(self.lin3(x)) + F.relu(self.lin4(x))

        return output

# GCN MODEL
class GCN(torch.nn.Module):
    name = "GCNConv"

    def __init__(self, in_channels:int = -1, hidden_channels:int = -1, out_channels:int = -1):
        super().__init__()
        assert in_channels > 0 and out_channels > 0, f"{GCN.name} model init was given bad args"
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.conv4 = GCNConv(hidden_channels, out_channels)
        self.bn1 = torch.nn.BatchNorm1d(num_features=hidden_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        edge_index = torch.stack([edge_index[1], edge_index[0]], dim=0)

        x = F.relu(self.bn1(self.conv1(x, edge_index)))
        output = F.relu(self.conv2(x, edge_index)) + F.relu(self.conv3(x, edge_index)) + F.relu(self.conv4(x, edge_index))

        return output

# EGCN MODEL
class EGCN(torch.nn.Module):
    name = "EGConv"

    def __init__(self, in_channels:int = -1, hidden_channels:int = -1, out_channels:int = -1, num_heads: int = 8, num_bases: int = 4):
        super().__init__()
        assert in_channels > 0 and out_channels > 0, f"{GCN.name} model init was given bad args"
        h_num_heads = num_heads if (hidden_channels % num_heads) == 0 else sys.exit(1) # That's a wrong config
        o_num_heads = num_heads if (out_channels % num_heads) == 0 else out_channels
        self.conv1 = EGConv(in_channels, hidden_channels, num_heads=h_num_heads, num_bases=h_num_heads // 2)
        self.conv2 = EGConv(hidden_channels, out_channels, num_heads=o_num_heads, num_bases=o_num_heads // 2)
        self.conv3 = EGConv(hidden_channels, out_channels, num_heads=o_num_heads, num_bases=o_num_heads // 2)
        self.conv4 = EGConv(hidden_channels, out_channels, num_heads=o_num_heads, num_bases=o_num_heads // 2)
        self.bn1 = torch.nn.BatchNorm1d(num_features=hidden_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        edge_index = torch.stack([edge_index[1], edge_index[0]], dim=0)

        x = F.relu(self.bn1(self.conv1(x, edge_index)))
        output = F.relu(self.conv2(x, edge_index)) + F.relu(self.conv3(x, edge_index)) + F.relu(self.conv4(x, edge_index))

        return output


# GRAPHCN MODEL
class GraphCN(torch.nn.Module):
    name = "GraphConv"

    def __init__(self, in_channels:int = -1, hidden_channels:int = -1, out_channels:int = -1):
        super().__init__()
        assert in_channels > 0 and out_channels > 0, f"{GraphCN.name} model init was given bad args"
        self.conv1 = GraphConv(in_channels, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, out_channels)
        self.conv3 = GraphConv(hidden_channels, out_channels)
        self.conv4 = GraphConv(hidden_channels, out_channels)
        self.bn1 = torch.nn.BatchNorm1d(num_features=hidden_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        edge_index = torch.stack([edge_index[1], edge_index[0]], dim=0)

        x = F.relu(self.bn1(self.conv1(x, edge_index)))
        output = F.relu(self.conv2(x, edge_index)) + F.relu(self.conv3(x, edge_index)) + F.relu(self.conv4(x, edge_index))

        return output


# GENCN MODEL
class GenCN(torch.nn.Module):
    name = "GENConv"

    def __init__(self, in_channels:int = -1, hidden_channels:int = -1, out_channels:int = -1):
        super().__init__()
        assert in_channels > 0 and out_channels > 0, f"{GenCN.name} model init was given bad args"
        self.conv1 = GENConv(in_channels, hidden_channels, num_layers=1, bias=True)
        self.conv2 = GENConv(hidden_channels, out_channels, num_layers=1, bias=True)
        self.conv3 = GENConv(hidden_channels, out_channels, num_layers=1, bias=True)
        self.conv4 = GENConv(hidden_channels, out_channels, num_layers=1, bias=True)
        self.bn1 = torch.nn.BatchNorm1d(num_features=hidden_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        edge_index = torch.stack([edge_index[1], edge_index[0]], dim=0)

        x = F.relu(self.bn1(self.conv1(x, edge_index)))
        output = F.relu(self.conv2(x, edge_index)) + F.relu(self.conv3(x, edge_index)) + F.relu(self.conv4(x, edge_index))

        return output


# GeneralCN MODEL
# -> Weird issue with running GeneralConv layers on cuda where collect() method has an assertion for non-None edge attributes ;)
class GeneralCN(torch.nn.Module):
    name = 'GeneralConv'

    def __init__(self, in_channels:int = -1, hidden_channels:int = -1, out_channels:int = -1):
        super().__init__()
        assert in_channels > 0 and out_channels > 0, f"{GeneralCN.name} model init was given bad args"
        self.conv1 = GeneralConv(in_channels, hidden_channels, in_edge_channels=1)
        self.conv2 = GeneralConv(hidden_channels, out_channels, in_edge_channels=1)
        self.conv3 = GeneralConv(hidden_channels, out_channels, in_edge_channels=1)
        self.conv4 = GeneralConv(hidden_channels, out_channels, in_edge_channels=1)
        self.bn1 = torch.nn.BatchNorm1d(num_features=hidden_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        edge_index = torch.stack([edge_index[1], edge_index[0]], dim=0)

        # Provide a dummy edge attribute (all zeros)
        edge_attr = torch.zeros(edge_index.size(1), 1).to(edge_index.device)

        x = F.relu(self.bn1(self.conv1(x, edge_index, edge_attr))) # Pass edge_attr to conv layers
        output = F.relu(self.conv2(x, edge_index, edge_attr)) + F.relu(self.conv3(x, edge_index, edge_attr)) + F.relu(self.conv4(x, edge_index, edge_attr))

        return output


# QGRN MODEL
import sys; sys.path.append(base_path)
from qgrl import QGRL
class QGRN(torch.nn.Module):
    name = 'QGRL'

    def __init__(self, in_channels:int = -1, hidden_channels:int = -1, out_channels:int = -1):
        super().__init__()
        assert in_channels > 0 and out_channels > 0, f"{QGRN.name} model init was given bad args"
        self.conv1 = QGRL(
                        in_channels=in_channels,
                        out_channels=hidden_channels,
                        num_sub_kernels=1,
                        normalize=False,
                        enable_activations=False,
                        quant_net_in_features=1,
                        quant_net_depth=2,
                        quant_net_expansion=1,
                        apply_mixture_net=False,
                        mixture_net_depth=1,
                        mixture_net_expansion=1,
                        apply_inner_resd_lyr=False)

        self.conv2 = QGRL(
                        in_channels=hidden_channels,
                        out_channels=out_channels,
                        num_sub_kernels=1,
                        normalize=False,
                        enable_activations=False,
                        quant_net_in_features=1,
                        quant_net_depth=2,
                        quant_net_expansion=1,
                        apply_mixture_net=False,
                        mixture_net_depth=1,
                        mixture_net_expansion=1,
                        apply_inner_resd_lyr=False)

        self.conv3 = QGRL(
                        in_channels=hidden_channels,
                        out_channels=out_channels,
                        num_sub_kernels=1,
                        normalize=False,
                        enable_activations=False,
                        quant_net_in_features=1,
                        quant_net_depth=2,
                        quant_net_expansion=1,
                        apply_mixture_net=False,
                        mixture_net_depth=1,
                        mixture_net_expansion=1,
                        apply_inner_resd_lyr=False)

        self.conv4 = QGRL(
                        in_channels=hidden_channels,
                        out_channels=out_channels,
                        num_sub_kernels=1,
                        normalize=False,
                        enable_activations=False,
                        quant_net_in_features=1,
                        quant_net_depth=2,
                        quant_net_expansion=1,
                        apply_mixture_net=False,
                        mixture_net_depth=1,
                        mixture_net_expansion=1,
                        apply_inner_resd_lyr=False)

        self.bn1 = torch.nn.BatchNorm1d(num_features=hidden_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        edge_index = torch.stack([edge_index[1], edge_index[0]], dim=0)

        x = F.relu(self.bn1(self.conv1(x, edge_index)))
        output = F.relu(self.conv2(x, edge_index)) + F.relu(self.conv3(x, edge_index)) + F.relu(self.conv4(x, edge_index))

        return output


In [ ]:
# Evaluates a model on a particular graph datasets x model
def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())

    return acc

# Trains a model on a particular graph datasets x model
def train_node_classifier(model, graph, optimizer, criterion, n_epochs=200, per_print_interval=10, step=1):
    print(f"\nStep: {step}")
    test_accs = [] # Assumes an early stopping mechanism
    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph)
        loss = criterion(out[graph.train_mask], graph.y[graph.train_mask])
        loss.backward()
        optimizer.step()

        train_acc = eval_node_classifier(model, graph, graph.train_mask)
        val_acc   = eval_node_classifier(model, graph, graph.val_mask)
        if epoch % per_print_interval == 0:
            print(f'Epoch: {epoch:03d}, Train Loss: {loss:.3f}, Train Acc: {train_acc:.3f}, Val Acc: {val_acc:.3f}')
        # Early stopping proxy below
        test_accs.append(eval_node_classifier(model, graph, graph.test_mask))

    return test_accs

# Process the results of the runs
def get_summary_of_run_results(results: dict, max_interested_range: int = 20):
    from statistics import mean, stdev

    summarized = {}
    for model_name, model_stats in results.items():
        sorted_stats = sorted(model_stats, reverse=True)
        trm_stats_range = sorted_stats[:max_interested_range]
        stats_min, stats_max, stats_mean, stats_stddev = trm_stats_range[-1], trm_stats_range[0], mean(trm_stats_range), stdev(trm_stats_range)
        summarized[model_name] = {
                                    "min": f'{(stats_min * 100):.2f} %',
                                    "max": f'{(stats_max * 100):.2f} %',
                                    "eff": f'{(stats_mean * 100):.2f} ± {(stats_stddev * 100):.2f}'
                                   }
    return summarized

def print_out_summary_results(summary_results: dict):
    print("\n=================================================================")
    print("Runs for models are summarized below:")
    print("=================================================================")
    for model_layer_name, model_stats in summary_results.items():
        print(f"\nModel: {model_layer_name}")
        for stat_name, stat_value in model_stats.items():
            print(f"\t {stat_name}: {stat_value}")


"""
Generic Helper function for training models
"""
def evaluate_models(models: list,
                    dataset:str = 'Cora',
                    lrs:list = [0.1, 0.01, 0.001],
                    max_epochs:int = 500,
                    num_randomizations:int=3,
                    d_split:dict={"num_val":0.2, "num_test":0.2},
                    model_hidden_channels: int = 32,
                    device:str='cpu:0'):

    # Helper to load the dataset with the right split
    def get_new_dataset_split(dataset:str = 'Cora'):
        # Loading dataset
        from torch_geometric.datasets import Planetoid
        from torch_geometric.datasets import WikipediaNetwork
        from torch_geometric.datasets import Actor
        from torch_geometric.datasets import HeterophilousGraphDataset
        from torch_geometric.datasets import Amazon, Reddit

        graph_dataset = None

        if dataset.strip().lower() == 'cora':
            graph_dataset = Planetoid(root=f'{base_path}/Datasets/Cora', name='Cora')
        elif dataset.strip().lower() == 'citeseer':
            graph_dataset = Planetoid(root=f'{base_path}/Datasets/CiteSeer', name='CiteSeer')
        elif dataset.strip().lower() == 'pubmed':
            graph_dataset = Planetoid(root=f'{base_path}/Datasets/PubMed', name='PubMed')
        elif dataset.strip().lower() == 'chameleon':
            graph_dataset = WikipediaNetwork(root=f'{base_path}/Datasets/Chameleon', name='Chameleon')
        elif dataset.strip().lower() == 'squirrel':
            graph_dataset = WikipediaNetwork(root=f'{base_path}/Datasets/Squirrel', name='Squirrel')
        elif dataset.strip().lower() == 'film':
            graph_dataset = Actor(root=f'{base_path}/Datasets/Film')
        elif dataset.strip().lower() == 'roman-empire':
            graph_dataset = HeterophilousGraphDataset(root=f'{base_path}/Datasets/RomanEmpire', name='Roman-empire')
        elif dataset.strip().lower() == 'amazon-computers':
            graph_dataset = Amazon(root=f'{base_path}Datasets/Amazon-Computers', name='Computers')
        elif dataset.strip().lower() == 'amazon-photo':
            graph_dataset = Amazon(root=f'{base_path}Datasets/Amazon-Photo', name='Photo')
        elif dataset.strip().lower() == 'reddit':
            graph_dataset = Reddit(root=f'{base_path}Datasets/Reddit')
        else:
            assert False, f"Dataset name = {dataset} not supported"

        return graph_dataset

    def split_dataset(graph_dataset, num_val:float = 0.2, num_test:float = 0.2):
        import torch_geometric.transforms as T
        from collections import Counter

        graph = T.RandomNodeSplit(num_val=num_val, num_test=num_test)(graph_dataset[0])

        print(f"\nDataset ({dataset}) stats:")
        print(f"Number of nodes: {graph_dataset[0].num_nodes}")
        print(f"Number of edges: {graph_dataset[0].num_edges}")
        print(f"Number of features: {graph_dataset[0].num_node_features}")
        print(f"Number of classes: {graph_dataset.num_classes}")
        print(f"Class Distribution: {Counter(graph.y.tolist()).items()}")

        return graph

    # Evaluate the models for the target set of conditions
    res = { model.name: [] for model in models }
    # load a different split: mimics a k-fold validation
    for i in range(num_randomizations):
        graph_dataset = get_new_dataset_split(dataset=dataset)
        graph = split_dataset(graph_dataset, num_val=d_split["num_val"], num_test=d_split["num_test"]).to(device)
        for j, lr in enumerate(lrs):
          for k, model in enumerate(models):
                mdl = model(in_channels=graph_dataset.num_node_features, hidden_channels=model_hidden_channels, out_channels=graph_dataset.num_classes).to(device)
                step = ((i * len(lrs) + j) * len(models)) + k
                criterion = nn.CrossEntropyLoss()
                optimizer_mdl = torch.optim.Adam(mdl.parameters(), lr=lr)
                test_accs = train_node_classifier(mdl, deepcopy(graph), optimizer_mdl, criterion, n_epochs=max_epochs, step=step)
                res[model.name].extend(test_accs)

                # Save the partial results
                results_path = f"{base_path}/Results"
                if not os.path.exists(results_path): os.mkdir(results_path)
                with open(f"{results_path}/{dataset.strip().lower()}.pk", "wb") as file_handle:
                  pickle.dump(res, file_handle, protocol=pickle.HIGHEST_PROTOCOL)

    # Post process the results
    res = { model.name: res[model.name] for model in models }

    # Return the results
    return res

In [ ]:
# Run experiment across Models x Apps
results = evaluate_models(models=[QGRN, GraphCN, GenCN, GeneralCN, EGCN],
                      dataset='PubMed', # Cora, CiteSeer, PubMed, Squirrel, Chameleon, Amazon-Computers, Amazon-Photo
                      lrs=[0.1, 0.05, 0.01, 0.005, 0.001],
                      max_epochs=2000,
                      num_randomizations=5,
                      d_split={"num_val":0.2, "num_test":0.2},
                      model_hidden_channels=64,
                      device=str(device))


In [ ]:
# # Print out result summary
with open(f"{base_path}/Results/pubmed.pk", "rb") as file_handle:
    results = pickle.load(file_handle)
print_out_summary_results(get_summary_of_run_results(results, max_interested_range=20))

In [ ]:
get_summary_of_run_results(results, max_interested_range=20)